In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [5]:
from notebook_init import *

out_root = Path('out/semantic')
makedirs(out_root, exist_ok=True)

# StyleGAN2 FFHQ

In [6]:
use_w = True
dataset = 'ffhq'
inst = get_instrumented_model('StyleGAN2', dataset, 'style', device, inst=inst, use_w=use_w)
model = inst.model
model.truncation = 1.0

gs_dir = np.load('./global_directions/ganspace_directions_ffhq.npy')#Note! Only ffhq is provided.
gs_dir = torch.from_numpy(gs_dir).to(device)

## GANSpace guided semantic

In [7]:
configs = [
#     (6, 8, 'ffhq', -20.0, 43, 'Disgusted', [140658858, 1887645531]),
#     (8, 9, 'ffhq', 9.0, 0, 'Makeup', [266415229, 375122892]),
#     (4, 5, 'ffhq', 10.0, 19, 'Big smile', [427229260]),
#     (6, 8, 'ffhq', -20.0, 33, 'Scary eyes', [1887645531]),
#     (2, 5, 'ffhq', 18.2, 21, 'Bald', [1635892780]),
#     (8, 9, 'ffhq', 13.0, 13, 'Bright BG vs FG', [798602383]),
#     (3, 6, 'ffhq', -60.0, 47, 'Curly hair', [1140578688]),
#     (8, 9, 'ffhq', -10.2, 16, 'Hair albedo', [427229260]),
#     (4, 7, 'ffhq', 10.0, 36, 'Displeased', [1887645531]),
#     (8, 9, 'ffhq', 20.0, 37, 'Eyebrow thickness', [1887645531]),
#     (7, 8, 'ffhq', -30.0, 54, 'Eye openness', [11573701]),
#     (0, 5, 'ffhq', 20.0, 37, 'Face roundness', [1887645531]),
#     (4, 10, 'ffhq', -20.0, 54, 'Fearful eyes', [11573701]),
#     (4, 5, 'ffhq', -13.6, 21, 'Hairline', [1635892780]),
#     (0, 8, 'ffhq', 20.0, 30, 'Happy frizzy hair', [1887645531]),
    (1, 4, 'ffhq', -10, 11, 'Eyeglasses', [140658858]), # paper
    (3, 6, 'ffhq', -5, 23, 'In awe', [1635892780]), # paper
#     (3, 6, 'ffhq', -5, 22, 'Large jaw', [1635892780]),
#     (10, 11, 'ffhq', 20.0, 34, 'Lipstick', [1887645531]),
#     (4, 5, 'ffhq', -30.0, 51, 'Nose length', [11573701]),
#     (8, 18, 'ffhq', 5.0, 27, 'Overexposed', [1887645531]),
#     (3, 7, 'ffhq', -14.5, 35, 'Screaming', [1887645531]),
    (2, 6, 'ffhq', -10, 32, 'Short face', [1887645531]), # paper
#     (4, 5, 'ffhq', -20.0, 46, 'Smile', [1175071341]),
#     (4, 5, 'ffhq', -20.0, 20, 'Unhappy bowl cut', [1635892780]),
    (8, 9, 'ffhq', -8, 10, 'Sunlight in face', [798602383]), # paper
#     (7, 9, 'ffhq', -40.0, 58, 'Trimmed beard', [1602858467]),
#     (3, 5, 'ffhq', -9.0, 20, 'Forehead hair', [1382206226]),
#     (0, 5, 'ffhq', -9.0, 21, 'Happy frizzy hair', [1382206226]),
#     (8, 9, 'ffhq', -15.0, 25, 'Light UD', [1382206226]),
#     (8, 11, 'ffhq', 9.0, 0, 'Makeup', [1953272274]),
#     (4, 6, 'ffhq', -16.0, 36, 'Smile', [1382206226]),
]

In [8]:
torch.autograd.set_grad_enabled(True)
img_save = 0
num_frames = 11
for config in configs:
    layer_start, layer_end, dataset, perturb_intensity, d, semantic, seeds = config
    for seed in seeds:
        print(seed)
        rng = np.random.RandomState(seed)
        noise, z, z_local_basis, z_sv, noise_basis = get_random_local_basis(model, rng)
        ''' rename & reshape'''    
        local_lat_comp = z_local_basis.t().unsqueeze(1).detach().to(device)
        local_lat_mean = z.detach().to(device)
        strips = []
        
        sim = compare_basis_componentwise(gs_dir[d], z_local_basis.t().cuda())[0]
        d_cp = torch.argmax(abs(sim))
        ori = torch.sign(sim[0][d_cp])
#         if semantic in ['Sunlight in face','Head angle up']:
#             perturb_intensity = perturb_intensity * 2
        batch_frames_gs = create_strip_centered(inst, 'latent', 'style', [z],
            0, gs_dir[d], 0, 1, 0, z, perturb_intensity, layer_start, layer_end, num_frames=num_frames)[0]
        batch_frames_cp = create_strip_centered(inst, 'latent', 'style', [z],
                            0, ori * local_lat_comp[d_cp], 0, 1, 0, z, perturb_intensity, layer_start, layer_end, num_frames=num_frames)[0]
        batch_frames_it = create_strip_iter(inst, 'latent', 'style', d_cp, sigma = perturb_intensity, layer_start = layer_start, layer_end = layer_end, 
                                             random_state = None, noise = noise, compare_basis = True, num_frames=num_frames, num_steps = 50, verbose=0)[0][0]
        strips.append(np.hstack(pad_frames(batch_frames_gs)))
        strips.append(np.hstack(pad_frames(batch_frames_cp)))
        strips.append(np.hstack(pad_frames(batch_frames_it)))

        grid = np.vstack(strips)
        if img_save:
            Image.fromarray(np.uint8(grid*255)).save(out_root / f'semantic_{seed}_layer{layer_start}to{layer_end}_{semantic}.jpg')
        else:
            print("GANSpace")
            print("Local Basis")
            print("Iterative Curve Traversal")
            plt.figure(figsize=(40,80))
            plt.imshow(grid)
            plt.axis('off')
            plt.show()
            print(f"semantic={semantic}, intensity={perturb_intensity}, cos_sim={sim[0][d_cp]}")
            print()
            

140658858
Getting iter trvs took 1.8077104091644287


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/isno/anaconda3/envs/local_basis/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_23268/1245839281.py", line 25, in <module>
    random_state = None, noise = noise, compare_basis = True, num_frames=num_frames, num_steps = 50, verbose=0)[0][0]
  File "/home/isno/localbasis/notebooks/localbasis_utils.py", line 243, in create_strip_iter
    layer_start, layer_end, compare_basis, num_frames, num_steps, only_pos, scale, verbose)
  File "/home/isno/localbasis/notebooks/localbasis_utils.py", line 252, in _create_strip_iter_impl
    layer_start, layer_end, compare_basis, num_frames, num_steps, only_pos, scale, verbose)
  File "/home/isno/localbasis/notebooks/localbasis_utils.py", line 288, in _create_strip_batch_sigma_iter
    img_batch = inst.model.sample_np(z)
  File "../models/wrappers.py", line 82, in sample_np
    img = self.forw

TypeError: object of type 'NoneType' has no len()